# Spam Filter

· Procedure
    
    · Divide data in train and test sets
    · Keep test data in a safe!    
    · Transform test data (normalize, discretize, etc)   
    · Train model    
    · Transform test data with the parameters found in step 3    
    · Test model with test data    
    · Evaluate results

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.stats import norm
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from random import random

In [2]:
df = pd.read_csv("spambase/spambase.data",header=None)
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


## Divide data in train and test sets

In [3]:
# Crear índices aleatorios para separar datos
index=np.array([1 if random() < 0.75 else 0 for i in range(len(df))])

# Crear conjunto train y test (sin saber si es spam)
X_train=np.array(df[df.columns[0:-1]])[index==1]
X_test=np.array(df[df.columns[0:-1]])[index==0]

# Guardar índices para saber si es spam
Y_train=np.array(df[df.columns[-1]])[index==1]
Y_test=np.array(df[df.columns[-1]])[index==0]

# Escalar los datos
scaler = preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

## Train Model

In [4]:
p_s = sum(Y_train)/float(len(Y_train))
p_ns = 1 - p_s
ln_ps= np.log(p_s)
ln_pns = np.log(p_ns)

In [5]:
mediaS = np.mean(X_train[Y_train==1], axis=0)
dsS = np.std(X_train[Y_train==1], axis=0)
mediaNS = np.mean(X_train[Y_train==0], axis=0)
dsNS = np.std(X_train[Y_train==0], axis=0)

## Function to detect class

In [6]:
def spamClass(obs):
    spam = ln_ps
    noSpam = ln_pns
    for i in range(len(obs)):
        if (mediaS[i] != 0.0 and dsS[i] > 0.0):
            spam = spam + np.log(norm(mediaS[i], dsS[i]).pdf(obs[i]))
        if (mediaNS[i] != 0.0 and dsNS[i] > 0.0):
            noSpam = noSpam + np.log(norm(mediaNS[i], dsNS[i]).pdf(obs[i]))
    if spam > noSpam:
        return 1
    else:
        return 0

## Test model with test data

In [7]:
pred = [spamClass(X_test[i]) for i in range(len(X_test))]

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


In [8]:
confusion_matrix(Y_test, pred)

array([[450, 235],
       [ 13, 434]])

In [9]:
# Con SKLearn
skl = GaussianNB()
skl.fit(X_train, Y_train) 
pred_skl = skl.predict(X_test) 
confusion_matrix(Y_test, pred_skl)

array([[479, 206],
       [ 18, 429]])

## Results

Con las matrices de confusión, podemos ver que el filtro creado tiene resultados similares al filtro utilizado de SKLearn. La mayoría de los mails los clasifica de manera correcta: spam-spam y nospam-nospam, por lo que podemos decir que el filtro funciona bien.